# Linear Regression with PyTorch
> This is a practice notebook for implementing linear regression in PyTorch.

- toc: true 
- badges: true
- comments: true
- categories: [pytorch]
- keyword: [ml, dl, linear, regression, pytorch]
- image: images/copied_from_nb/images/2022-10-10-pytorch-linear-regression.jpeg

![](images/2022-10-10-pytorch-linear-regression.jpeg)

# Introduction

In this notebook we will train a linear regression model using PyTorch.

# Environment
This notebook is prepared with Google Colab.

# Credits
This notebook takes inspiration from book "Deep Learning with PyTorch Step-by-Step" by "Daniel Voigt Godoy". Official GitHub repository for this book has a useful notebooks: https://github.com/dvgodoy/PyTorchStepByStep


# prepare linear data
import numpy as np
np.random.seed(0)

"""
y = mx + c + e
"""


In [12]:

def generate_linear_data(n_data_points=100, true_m=1, true_b=1):
    x = np.random.rand(n_data_points, 1)
    y = true_m * x + true_b + (.1 * np.random.randn(n_data_points, 1))
    return x, y


n_data_points = 100
true_m = 2
true_b = 1
x, y = generate_linear_data(n_data_points, true_m, true_b)

In [ ]:
# plot generated data
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.scatter(x, y)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Generated Full Dataset')
plt.show()

In [ ]:
# convert data to torch tensors

import torch

x_tensor = torch.as_tensor(x).float()
y_tensor = torch.as_tensor(y).float()

In [ ]:
# load data into DataLoader
from torch.utils.data import DataLoader, TensorDataset, random_split

dataset = TensorDataset(x_tensor, y_tensor)

# Performs the split
ratio = .8
n_total = len(dataset)
n_train = int(n_total * ratio)
n_val = n_total - n_train

train_data, val_data = random_split(dataset, [n_train, n_val])

# Builds a loader of each set
train_loader = DataLoader(
    dataset=train_data,
    batch_size=16,
    shuffle=True
)
val_loader = DataLoader(dataset=val_data, batch_size=16)

In [ ]:
# visualize train and val data

# get data from dataset
train_data_list = list(iter(train_data))
val_data_list = list(iter(val_data))

# get x,y from dataset tuples list
x_train = [e[0].numpy() for e in train_data_list]
y_train = [e[1].numpy() for e in train_data_list]

x_val = [e[0].numpy() for e in val_data_list]
y_val = [e[1].numpy() for e in val_data_list]

# plot data
figure, axes = plt.subplots(1, 3, figsize=(10,5))
figure.suptitle('Train and Validation Dataset')

axes[0].set_title('Training Data')
axes[0].scatter(x_train, y_train)

axes[1].set_title('Validation Data')
axes[1].scatter(x_val, y_val)

axes[2].set_title('Combined Data')
axes[2].scatter(x_train, y_train)
axes[2].scatter(x_val, y_val)
plt.show()

In [ ]:
# Model configuration

import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Sets learning rate - this is "eta" ~ the "n" like Greek letter
lr = 0.1

torch.manual_seed(0)
# Now we can create a model and send it at once to the device
model = nn.Linear(1, 1).to(device)

# Defines a SGD optimizer to update the parameters (now retrieved directly from the model)
optimizer = optim.SGD(model.parameters(), lr=lr)

# Defines a MSE loss function
loss_fn = nn.MSELoss(reduction='mean')

In [ ]:
# weights before training
print(model.state_dict())

In [ ]:
# training pipeline

## helper function for training
def make_train_step_fn(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def perform_train_step_fn(x, y):
        # Sets model to TRAIN mode
        model.train()
        
        # Step 1 - Computes our model's predicted output - forward pass
        yhat = model(x)
        # Step 2 - Computes the loss
        loss = loss_fn(yhat, y)
        # Step 3 - Computes gradients for both "a" and "b" parameters
        loss.backward()
        # Step 4 - Updates parameters using gradients and the learning rate
        optimizer.step()
        optimizer.zero_grad()
        
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return perform_train_step_fn

In [ ]:
## helper function for validation
def make_val_step_fn(model, loss_fn):
    # Builds function that performs a step in the validation loop
    def perform_val_step_fn(x, y):
        # Sets model to EVAL mode
        model.eval()
        
        # Step 1 - Computes our model's predicted output - forward pass
        yhat = model(x)
        # Step 2 - Computes the loss
        loss = loss_fn(yhat, y)
        # There is no need to compute Steps 3 and 4, since we don't update parameters during evaluation
        return loss.item()
    
    return perform_val_step_fn

In [ ]:
import datetime

# Creates the train_step function for our model, loss function and optimizer
train_step_fn = make_train_step_fn(model, loss_fn, optimizer)

# Creates the val_step function for our model and loss function
val_step_fn = make_val_step_fn(model, loss_fn)

# Creates a Summary Writer to interface with TensorBoard
timestamp = datetime.datetime.utcnow().strftime('%Y-%m-%d-%H.%M.%S')
writer = SummaryWriter(f'runs/simple_linear_regression/{timestamp}')

# Fetches a single mini-batch so we can use add_graph
x_sample, y_sample = next(iter(train_loader))
writer.add_graph(model, x_sample.to(device))

In [ ]:
# helper function for minibatch processing
def mini_batch(device, data_loader, step_fn):
    mini_batch_losses = []
    for x_batch, y_batch in data_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        mini_batch_loss = step_fn(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)

    loss = np.mean(mini_batch_losses)
    return loss

In [ ]:
# execute pipeline with training and validation steps

n_epochs = 200

losses = []
val_losses = []

for epoch in range(n_epochs):
    # inner loop
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)
    
    # VALIDATION
    # no gradients in validation!
    with torch.no_grad():
        val_loss = mini_batch(device, val_loader, val_step_fn)
        val_losses.append(val_loss)
    
    # Records both losses for each epoch under the main tag "loss"
    writer.add_scalars(main_tag='loss',
                       tag_scalar_dict={'training': loss, 'validation': val_loss},
                       global_step=epoch)
    
    print(f"epoch: {epoch:3}, train loss: {loss:.5f}, valid loss: {val_loss:.5f}")

# Closes the writer
writer.close()

In [ ]:
# model weights after training
print(model.state_dict())

In [ ]:
f'runs/simple_linear_regression/{timestamp}'

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(x_train, y_train)

In [ ]:
reg.coef_, reg.intercept_